- Import the create_engine function from the sqlalchemy library.
- Create an engine to the census database by concatenating the following strings and passing them to create_engine():
- 'mysql+pymysql://' (the dialect and driver).
- 'student:datacamp' (the username and password).
- '@courses.csrrinzqubik.us-east-1.rds.amazonaws.com:3306/' (the host and port).
- 'census' (the database name).
- Use the .table_names() method on engine to print the table names.

In [2]:
# # Import create_engine function
# from sqlalchemy import create_engine

# # Create an engine to the census database
# engine = create_engine('mysql+pymysql://' + 'student:datacamp'+ '@courses.csrrinzqubik.us-east-1.rds.amazonaws.com:3306/' +'census' )

# # Print the table names
# print(engine.table_names())


- Define a select statement called stmt to return:
- i) The state column of the census table (census.columns.state).
- ii) The difference in population count between 2008 (census.columns.pop2008) and 2000 (census.columns.pop2000) labeled as 'pop_change'.
- Group the statement by census.columns.state.
- Order the statement by population change ('pop_change') in descending order. Do so by passing it desc('pop_change').
- Use the .limit() method on the previous statement to return only 5 records.
- Execute the statement and fetchall() the records.
- The print statement has already been written for you. Hit 'Submit Answer' to view the results!

In [6]:
from sqlalchemy import func,desc
# Import create_engine, MetaData, and Table
from sqlalchemy import create_engine, MetaData, Table,select

# Create engine: engine
engine = create_engine('sqlite:///census.sqlite')

# Create a metadata object: metadata
metadata = MetaData()

# Reflect census table from the engine: census
census = Table('census', metadata, autoload=True, autoload_with=engine)

# Create a connection on engine
connection = engine.connect()

# Create a select query: stmt
stmt = select([census])

# Build query to return state names by population difference from 2008 to 2000: stmt
stmt = select([census.columns.state, (census.columns.pop2008-census.columns.pop2000).label('pop_change')])

# Append group by for the state: stmt_grouped
stmt_grouped = stmt.group_by(census.columns.state)

# Append order by for pop_change descendingly: stmt_ordered
stmt_ordered = stmt_grouped.order_by(desc('pop_change'))

# Return only 5 results: stmt_top5
stmt_top5 = stmt_ordered.limit(5)

# Use connection to execute stmt_top5 and fetch all results
results = connection.execute(stmt_top5).fetchall()

# Print the state and population change for each record
for result in results:
    print('{}:{}'.format(result.state, result.pop_change))


Texas:40137
California:35406
Florida:21954
Arizona:14377
Georgia:13357


- Import case, cast, and Float from sqlalchemy.
- Build an expression female_pop2000to calculate female population in 2000. To achieve this:
- Use case() inside func.sum().
- The first argument of case() is a list containing a tuple of
- i) A boolean checking that census.columns.sex is equal to 'F'.
- ii) The column census.columns.pop2000.
- The second argument is the else_ condition, which should be set to 0.
- Calculate the total population in 2000 and use cast() to convert it to Float.
- Build a query to calculate the percentage of women in 2000. To do this, divide female_pop2000 by total_pop2000 and multiply by 100.
- Execute the query and print percent_female.

In [7]:
# import case, cast and Float from sqlalchemy
from sqlalchemy import case, cast, Float

# Build an expression to calculate female population in 2000
female_pop2000 = func.sum(
    case([
        (census.columns.sex == 'F', census.columns.pop2000)
    ], else_=0))

# Cast an expression to calculate total population in 2000 to Float
total_pop2000 = cast(func.sum(census.columns.pop2000), Float)

# Build a query to calculate the percentage of women in 2000: stmt
stmt = select([female_pop2000 / total_pop2000 * 100])

# Execute the query and store the scalar result: percent_female
percent_female = connection.execute(stmt).scalar()

# Print the percentage
print(percent_female)


51.09467432293413


- Build a statement to join the census and state_fact tables and select the pop2000 column from the first and the abbreviation column from the second.
- Execute the statement to get the first result and save it as result.
- Hit 'Submit Answer' to loop over the keys of the result object, and print the key and value for each!

In [9]:
# # Build a statement to join census and state_fact tables: stmt
# stmt = select([census.columns.pop2000, state_fact.columns.abbreviation])

# # Execute the statement and get the first result: result
# result = connection.execute(stmt).first()

# # Loop over the keys in the result object and print the key and value
# for key in result.keys():
#     print(key, getattr(result, key))


- Build a statement to select ALL the columns from the census and state_fact tables. To select ALL the columns from two tables employees and sales, for example, you would use stmt = select([employees, sales]).
- Append a select_from to stmt to join the census table to the state_fact table by the state column in census and the name column in the state_fact table.
- Execute the statement to get the first result and save it as result. This code is already written.
- Hit 'Submit Answer' to loop over the keys of the result object, and print the key and value for each!

In [10]:
# # Build a statement to select the census and state_fact tables: stmt
# stmt = select([census, state_fact])

# # Add a select_from clause that wraps a join for the census and state_fact
# # tables where the census state column and state_fact name column match
# stmt_join = stmt.select_from(
#     census.join(state_fact, census.columns.state == state_fact.columns.name))

# # Execute the statement and get the first result: result
# result = connection.execute(stmt_join).first()

# # Loop over the keys in the result object and print the key and value
# for key in result.keys():
#     print(key, getattr(result, key))


- Build a statement to select:
- The state column from the census table.
- The sum of the pop2008 column from the census table.
- The census_division_name column from the state_fact table.
- Append a .select_from() to stmt in order to join the census and state_fact tables by the state and name columns.
- Group the statement by the name column of the state_fact table.
- Execute the statement stmt_grouped to get all the records and save it as results.
- Hit 'Submit Answer' to loop over the results object and print each record.

In [11]:
# # Build a statement to select the state, sum of 2008 population and census
# # division name: stmt
# stmt = select([
#     census.columns.state,
#     func.sum(census.columns.pop2008),
#     state_fact.columns.census_division_name
# ])

# # Append select_from to join the census and state_fact tables by the census state and state_fact name columns
# stmt_joined = stmt.select_from(
#     census.join(state_fact, census.columns.state == state_fact.columns.name)
# )

# # Append a group by for the state_fact name column
# stmt_grouped = stmt_joined.group_by(state_fact.columns.name)

# # Execute the statement and get the results: results
# results = connection.execute(stmt_grouped).fetchall()

# # Loop over the results object and print each record.
# for record in results:
#     print(record)


- Save an alias of the employees table as managers. To do so, apply the method .alias() to employees.
- Build a query to select the employee's name and their manager's name. The manager's name has already been selected for you. Use label to label the name column of employees as 'employee'.
- Append a where clause to stmt to match where the id column of the managers table corresponds to the mgr column of the employees table.
- Order the statement by the name column of the managers table.
- Execute the statement and store all the results. This code is already written. Hit 'Submit Answer' to print the names of the managers and all their employees.

In [15]:
# Import create_engine, MetaData, and Table
from sqlalchemy import create_engine, MetaData, Table,select

# Create engine: engine
engine = create_engine('sqlite:///employees.sqlite')

# Create a metadata object: metadata
metadata = MetaData()

# Reflect census table from the engine: census
employees = Table('employees', metadata, autoload=True, autoload_with=engine)

# Create a connection on engine
connection = engine.connect()

# Create a select query: stmt
employees = select([employees])

In [16]:
# Make an alias of the employees table: managers
managers = employees.alias()

# Build a query to select names of managers and their employees: stmt
stmt = select(
    [managers.columns.name.label('manager'),
     employees.columns.name.label('employee')]
)

# Match managers id with employees mgr: stmt_matched
stmt_matched = stmt.where(managers.columns.id == employees.columns.mgr)

# Order the statement by the managers name: stmt_ordered
stmt_ordered = stmt_matched.order_by(managers.columns.name)

# Execute statement: results
results = connection.execute(stmt_ordered).fetchall()

# Print records
for record in results:
    print(record)



('FILLMORE', 'GRANT')
('FILLMORE', 'ADAMS')
('FILLMORE', 'MONROE')
('GARFIELD', 'JOHNSON')
('GARFIELD', 'LINCOLN')
('GARFIELD', 'POLK')
('GARFIELD', 'WASHINGTON')
('HARDING', 'TAFT')
('HARDING', 'HOOVER')
('JACKSON', 'HARDING')
('JACKSON', 'GARFIELD')
('JACKSON', 'FILLMORE')
('JACKSON', 'ROOSEVELT')


C:\Users\88016\AppData\Local\Temp/ipykernel_30764/4100943118.py:7: SADeprecationWarning: The SelectBase.c and SelectBase.columns attributes are deprecated and will be removed in a future release; these attributes implicitly create a subquery that should be explicit.  Please call SelectBase.subquery() first in order to create a subquery, which then contains this attribute.  To access the columns that this SELECT object SELECTs from, use the SelectBase.selected_columns attribute. (deprecated since: 1.4)
  employees.columns.name.label('employee')]
C:\Users\88016\AppData\Local\Temp/ipykernel_30764/4100943118.py:11: SADeprecationWarning: The SelectBase.c and SelectBase.columns attributes are deprecated and will be removed in a future release; these attributes implicitly create a subquery that should be explicit.  Please call SelectBase.subquery() first in order to create a subquery, which then contains this attribute.  To access the columns that this SELECT object SELECTs from, use the Sele

- Save an alias of the employees table as managers.
- Build a query to select the name column of the managers table and the count of the number of their employees. The function func.count() has been imported and will be useful! Use it to count the id column of the employees table.
- Using a .where() clause, filter the records where the id column of the managers table and mgr column of the employees table are equal.
- Group the query by the name column of the managers table.
- Execute the statement and store all the results. Print the names of the managers and their employees. This code has already been written so hit 'Submit Answer' and check out the results!

In [17]:
# Make an alias of the employees table: managers
managers = employees.alias()

# Build a query to select names of managers and counts of their employees: stmt
stmt = select([managers.columns.name, func.count(employees.columns.id)])

# Append a where clause that ensures the manager id and employee mgr are equal
stmt_matched = stmt.where(employees.columns.mgr == managers.columns.id)

# Group by Managers Name
stmt_grouped = stmt_matched.group_by(managers.columns.name)

# Execute statement: results
results = connection.execute(stmt_grouped).fetchall()

# print manager
for record in results:
    print(record)



('FILLMORE', 3)
('GARFIELD', 4)
('HARDING', 2)
('JACKSON', 4)


C:\Users\88016\AppData\Local\Temp/ipykernel_30764/2530089928.py:5: SADeprecationWarning: The SelectBase.c and SelectBase.columns attributes are deprecated and will be removed in a future release; these attributes implicitly create a subquery that should be explicit.  Please call SelectBase.subquery() first in order to create a subquery, which then contains this attribute.  To access the columns that this SELECT object SELECTs from, use the SelectBase.selected_columns attribute. (deprecated since: 1.4)
  stmt = select([managers.columns.name, func.count(employees.columns.id)])
C:\Users\88016\AppData\Local\Temp/ipykernel_30764/2530089928.py:8: SADeprecationWarning: The SelectBase.c and SelectBase.columns attributes are deprecated and will be removed in a future release; these attributes implicitly create a subquery that should be explicit.  Please call SelectBase.subquery() first in order to create a subquery, which then contains this attribute.  To access the columns that this SELECT obj

- Use a while loop that checks if there are more_results.
- Inside the loop, apply the method .fetchmany() to results_proxy to get 50 records at a time and store those records as partial_results.
- After fetching the records, if partial_results is an empty list (that is, if it is equal to []), set more_results to False.
- Loop over the partial_results and, if row.state is a key in the state_count dictionary, increment state_count[row.state] by 1; otherwise set state_count[row.state] to 1.
- After the while loop, close the ResultProxy results_proxy using .close().
- Hit 'Submit Answer' to print state_count.

In [1]:
# # Start a while loop checking for more results
# while more_results:
#     # Fetch the first 50 results from the ResultProxy: partial_results
#     partial_results = results_proxy.fetchmany(50)

#     # if empty list, set more_results to False
#     if partial_results == []:
#         more_results = False

#     # Loop over the fetched records and increment the count for the state
#     for row in partial_results:
#         if row.state in state_count:
#             state_count[row.state] = state_count[row.state] + 1
#         else:
#             state_count[row.state] = 1

# # Close the ResultProxy, and thus the connection
# results_proxy.close()

# # Print the count by state
# print(state_count)